#Fine-tune AraBERT with Fast-BERT library
https://github.com/kaushaltrivedi/fast-ber

##Get Requirements

In [1]:
!git clone https://github.com/aub-mind/arabert
!pip install PyArabic farasapy fast-bert

Cloning into 'arabert'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 279 (delta 34), reused 38 (delta 15), pack-reused 214
Receiving objects: 100% (279/279), 3.68 MiB | 9.82 MiB/s, done.
Resolving deltas: 100% (157/157), done.
     |████████████████████████████████| 91 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 43 kB 1.2 MB/s eta 0:00:011


     |████████████████████████████████| 3.0 MB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 769 kB 12.6 MB/s eta 0:00:01
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16170 sha256=14b3cc34321d6a7136b7eda5c47cfd809f7f833262fb72b75cf91e4b66f3ad0d
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.9.3
    Uninstalling tokenizers-0.9.3:
      Successfully uninstalled tokenizers-0.9.3
  Attempting uninstall: transformers
    Found existing installation: transformers 3.5.1
    Uninstalling transformers-3.5.1:
      Successfully uninstalled transformers-3.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 1.2.2 requires transformers<3.6,>=3.4, but you ha

##Prepare the data

In [18]:

df_AJGT = pd.read_csv('../input/tunizi-icompass/tunizi_train.txt')
df_AJGT

,ID,text,label
0,21037,alah yara7me,1
1,46442,brabi atini najah wahed amalta fi akaber korat...,1
2,45602,bravo slouma walah rajel,1
3,30855,elboutoula ma nefhem chay,1
4,19151,ma7laa zinkk,1
...,...,...,...
69995,28436,amazing i love it,1
69996,43069,ena raniiiiiiiiiii m3a eriahi ye3niii m3a jam3...,1
69997,68137,hada maycharafech tounis,-1
69998,95143,leken rajlek haw et7adek wemcher we3mel inteviou,-1


In [2]:
import pandas as pd
#from py4j.java_gateway import JavaGateway
from farasa.segmenter import FarasaSegmenter
from arabert.preprocess_arabert import preprocess
from sklearn.model_selection import train_test_split

# !pkill "java"
# gateway = JavaGateway.launch_gateway(classpath='./FarasaSegmenterJar.jar')
# farasa_segmenter = gateway.jvm.com.qcri.farasa.segmenter.Farasa()

farasa_segmenter = FarasaSegmenter(interactive=True)



#df_AJGT = pd.read_csv('../input/sentiment/Train.csv',header=0)

DATA_COLUMN = 'text'
LABEL_COLUMN = 'label'

df_AJGT = df_AJGT[['text', 'label']]

df_AJGT.columns = [DATA_COLUMN, LABEL_COLUMN]



df_AJGT[DATA_COLUMN] = df_AJGT[DATA_COLUMN].apply(lambda x: preprocess(x, do_farasa_tokenization=True , farasa=farasa_segmenter, use_farasapy = True))
# df_AJGT[LABEL_COLUMN] = df_AJGT[LABEL_COLUMN].apply(lambda x: label_map[x])
df_AJGT['label']=df_AJGT['label'].astype(str)
train_AJGT, test_AJGT = train_test_split(df_AJGT, test_size=0.3,random_state=42)
!mkdir data
train_AJGT.to_csv("data/train.csv",index=True,columns=train_AJGT.columns,sep=',',header=True)
test_AJGT.to_csv("data/dev.csv",index=True,columns=test_AJGT.columns,sep=',',header=True)
with open('data/labels.csv','w') as f:
    f.write("\n".join(df_AJGT['label'].unique()))

/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


100%|██████████| 241M/241M [00:15<00:00, 13.7MiB/s] 

[2021-01-01 10:22:58,788 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


TypeError: string indices must be integers

In [ ]:
df_AJGT

In [ ]:
with open('data/labels.csv','w') as f:
    f.write("\n".join(df_AJGT['label'].unique()))

##Create a DataBunch Object:
see https://github.com/kaushaltrivedi/fast-bert#text-classification

In [ ]:
from fast_bert.data_cls import BertDataBunch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('asafaya/bert-base-arabic')

databunch = BertDataBunch('./data/', './data/',
                          tokenizer=tokenizer,
                          train_file='train.csv',
                          val_file='dev.csv',
                          label_file='labels.csv',
                          text_col='text',
                          label_col='label',
                          batch_size_per_gpu=16,
                          max_seq_length=256,
                          multi_gpu=True,
                          multi_label=False,
                          model_type='bert',
                          )

##Create the Learner Object
see https://github.com/kaushaltrivedi/fast-bert#2-create-a-learner-object

In [ ]:
import logging
import torch

from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

device_cuda = torch.device("cuda")
metrics = [{'name': 'accuracy', 'function': accuracy}]

!mkdir 'output'
learner = BertLearner.from_pretrained_model(
						databunch,
						pretrained_path='asafaya/bert-base-arabic',
						metrics=metrics,
						device=device_cuda,
						logger=logger,
						output_dir='output',
						finetuned_wgts_path=None,
						warmup_steps=30,
						multi_gpu=False,
						is_fp16=False,
						multi_label=False,
						logging_steps=0)

##Start Training and Validating

In [ ]:
learner.fit(epochs=3,
			lr=2e-5,
			validate=True, 	# Evaluate the model after each epoch
			schedule_type="warmup_linear",
			optimizer_type="adamw")

##You can see the output using tensorboard

In [ ]:
learner.save_model()

In [ ]:
metrics

In [ ]:
#%load_ext tensorboard
#%tensorboard --logdir './output/tensorboard'

In [ ]:
from fast_bert.modeling import BertForMultiLabelSequenceClassification
from fast_bert.data import BertDataBunch, InputExample, InputFeatures, MultiLabelTextProcessor, convert_examples_to_features
#from fast_bert.learner import BertLearner
from fast_bert.metrics import accuracy_multilabel, accuracy_thresh, fbeta, roc_auc

In [ ]:
from functools import partial

metrics = []
#metrics.append({'name': 'accuracy_thresh', 'function': accuracy_thresh})
metrics.append({'name': 'roc_auc', 'function': roc_auc})
metrics.append({'name': 'F1', 'function': partial(fbeta, beta=1)})
metrics.append({'name': 'accuracy_single', 'function': accuracy_multilabel})


In [ ]:
#Predictions
#Call predict_batch method on the learner object that contains the trained model.

texts = [
  "مخيب عيشتكم وملا حالة هالكورونا",
  "البلاد مشات يا لطيف"
]

predictions = learner.predict_batch(texts)
from fast_bert.prediction import BertClassificationPredictor

predictor = BertClassificationPredictor('asafaya/bert-base-arabic','./data/')

# Single prediction
single_prediction = predictor.predict("مخيب عيشتكم وملا حالة هالكورونا")

# Batch predictions
texts = [
  "مخيب عيشتكم وملا حالة هالكورونا",
  "البلاد مشات يا لطيف"
]

multiple_predictions = predictor.predict(texts)

In [ ]:
multiple_predictions

In [ ]:
single_prediction = predictor.predict("Revoyez vos prix ainsi que la lenteur ke prend lapp pr se connecter.")
single_prediction

In [ ]:

single_prediction = predictor.predict("قمة الإستفزاز و النذالة في هالبلاد إنو في برنامج في تلفزة كيما التونسية يجيبو واحد طحان متع بن علي ...")
single_prediction

In [ ]:
output_asafaya = predictor.predict_batch(list(pd.read_csv("../input/sentiment/Test.csv")['content'].values))

In [ ]:
preds = pd.DataFrame([{item[0]: item[1] for item in pred} for pred in output_asafaya])

In [ ]:
test_df = pd.read_csv('../input/sentiment/Test.csv',header=0)

In [ ]:
output_df = pd.merge(test_df, preds, how='left', left_index=True, right_index=True)


In [ ]:

columns = ['ID','-1','0','1']

In [ ]:
output_df = output_df[columns]

In [ ]:

output_df.to_csv('asafaya.csv', index=None)

100%|██████████| 241M/241M [00:30<00:00, 13.7MiB/s]